# Verify the data scientist role


#### Prerequirement

* For demostaration purposed, Let jupyternotebook SageMaker execution role to perform `sts:AssumeRole` on resource any.


In [1]:
import boto3

In [2]:
# get current accout and region
region = boto3.Session().region_name
account = boto3.client('sts').get_caller_identity().get('Account')
roleArn = f'arn:aws:iam::{account}:role/abc.dev.{region}.data-scientist.basic-execution.role'
roleSSName = 'dev.data-science'

sagemaker_nb_role = f'arn:aws:iam::{account}:role/abc.dev.all.sagemaker.data-scientist-exec.role'

In [3]:
sts = boto3.client('sts')

response = sts.assume_role(RoleArn=roleArn, RoleSessionName=roleSSName)
ds_credentials = response['Credentials']

In [4]:
# use data-science credentials to create sagemaker
ds_session = boto3.Session(aws_access_key_id=ds_credentials['AccessKeyId'],
    aws_secret_access_key=ds_credentials['SecretAccessKey'],
    aws_session_token=ds_credentials['SessionToken'],
    region_name=region)

In [5]:
ds_sm = ds_session.client(service_name='sagemaker')

In [9]:
from datetime import datetime
now = datetime.now().strftime("-%Y-%m-%d-%H-%M-%S") # current date and time

nb_name = 'demo' + now

In [11]:
response = ds_sm.create_notebook_instance(NotebookInstanceName=nb_name,
                                          InstanceType='ml.t3.medium',
                                          RoleArn=sagemaker_nb_role)
response

{'NotebookInstanceArn': 'arn:aws:sagemaker:us-west-2:752257916058:notebook-instance/demo-2020-07-27-13-18-07',
 'ResponseMetadata': {'RequestId': 'b3c1bf8a-3795-4230-9818-94db2d5d99ef',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'b3c1bf8a-3795-4230-9818-94db2d5d99ef',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '109',
   'date': 'Mon, 27 Jul 2020 13:18:24 GMT'},
  'RetryAttempts': 0}}

In [10]:
response = ds_sm.create_notebook_instance(NotebookInstanceName=nb_name,
                                          InstanceType='ml.p3.2xlarge',
                                          RoleArn=sagemaker_nb_role)
response

ClientError: An error occurred (AccessDeniedException) when calling the CreateNotebookInstance operation: User: arn:aws:sts::752257916058:assumed-role/abc.dev.us-west-2.data-scientist.basic-execution.role/dev.data-science is not authorized to perform: sagemaker:CreateNotebookInstance on resource: arn:aws:sagemaker:us-west-2:752257916058:notebook-instance/demo-2020-07-27-13-18-07